In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
import sklearn
from sklearn.pipeline import Pipeline

stopwords_rus = stopwords.words('russian')

In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [3]:
from tqdm import tqdm

In [4]:
metadata = pd.read_csv('meta_rubrics_final.tsv', encoding = 'utf-8', sep = '\t')

In [5]:
def preprocess(texts):
    preprocessed_texts_list = []
    for text in tqdm(texts):
        #del_new_line = re.sub(r'\n', '', text.lower()) 
        extracted_text = re.findall(r'[\w]+', text.lower())#разделяем на токены
        extracted_text = ' '.join(extracted_text)
        preprocessed_texts_list.append(extracted_text)
    return preprocessed_texts_list

In [8]:
articles = pd.DataFrame(data = list(zip(metadata.path, metadata.title, metadata.tags, metadata.final_rubrics, metadata.number_of_rubrics)), 
                        columns = ['path', 'title', 'tags', 'rubrics', 'rubrics_number'])

In [9]:
articles.head()

,path,title,tags,rubrics,rubrics_number
0,chrdk.ru/articles/sci_10_salt_lakes.txt,Десять самых известных соленых озер,География_Экология,Науки о земле,One
1,chrdk.ru/articles/sci_33_fractures.txt,33 перелома,Российская наука_Антропология,История,One
2,chrdk.ru/articles/sci_46_chromosomes.txt,46 — норма?,Генетика_Медицина,Физиология человека,One
3,chrdk.ru/articles/sci_750gev.txt,Несбывшиеся надежды на новую физику,Физика_Интервью_Закрытия,Физика,One
4,chrdk.ru/articles/sci_alien_anatomy.txt,Анатомия каменных пришельцев,Геология_Космос,Космос,Multi


In [29]:
articles_sample = articles[articles.rubrics != 'Мусор'].head(2000)

In [30]:
texts = []
for path in tqdm(articles_sample.path):
    path = path.replace('\n','').replace('//','/').replace('?code=','-code=')
    try:
        with open('C:/Users/Анна/YandexDisk/popular_science_texts_store/' + path, encoding = 'utf-8') as f:
            texts.append(f.read())
    except OSError:
        texts.append('')

100%|████████████████████████████████████| 2000/2000 [00:01<00:00, 1307.13it/s]


In [39]:
metadata[metadata.path == 'nplus1.ru/nplus1_news/nplus1.ru-news-2016-01-15-dark-side-moon?code=1dad93b27f46ea76816e3938f188885d.txt\n']

,path,source,url,date,title,subtitle,author,tags,genre,mapped_rubrics,final_rubrics
11962,nplus1.ru/nplus1_news/nplus1.ru-news-2016-01-1...,nplus1.ru,nplus1.ru/news/2016/01/21/gambling,2016-01-21,Светомузыка сделала крыс азартными игроками,NaN,Олег Лищук,_Наука_,Новости,Мусор,Мусор


In [ ]:
#имена файлов
#индикатор - путь начинается с /
#nplus1 - все имена заканчиваются на \n, 
#вместо ...moon-code=1dad93b27f46ea76816e3938f188885d -> moon?code=1dad93b27f46ea76816e3938f188885d

In [31]:
articles_sample['text'] = texts

In [32]:
articles_sample.head()

,path,title,tags,rubrics,rubrics_number,text
0,chrdk.ru/articles/sci_10_salt_lakes.txt,Десять самых известных соленых озер,География_Экология,Науки о земле,One,"Возможно, не все об этом знают, но объемы воды..."
1,chrdk.ru/articles/sci_33_fractures.txt,33 перелома,Российская наука_Антропология,История,One,Останки мужчины с зажившими переломами нашли п...
2,chrdk.ru/articles/sci_46_chromosomes.txt,46 — норма?,Генетика_Медицина,Физиология человека,One,"В отличие от зубов, хромосом человеку положено..."
3,chrdk.ru/articles/sci_750gev.txt,Несбывшиеся надежды на новую физику,Физика_Интервью_Закрытия,Физика,One,"В начале августа CERN официально объявил, что ..."
4,chrdk.ru/articles/sci_alien_anatomy.txt,Анатомия каменных пришельцев,Геология_Космос,Космос,Multi,Как выглядят и чем отличаются друг от друга го...


In [33]:
articles_sample.text = preprocess(articles_sample.text)

100%|████████████████████████████████████| 2000/2000 [00:01<00:00, 1018.81it/s]


In [34]:
articles_sample.text[0][:250]

'возможно не все об этом знают но объемы воды в пресных и соленых озерах на планете примерно одинаковы а вот научное внимание распределено неравномерно исследований в области пресных водоемов в несколько раз больше во многом это удивительно ведь солен'

In [36]:
len(set(articles_sample.rubrics))

16

In [41]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import *
from sklearn.pipeline import *
from sklearn.preprocessing import Normalizer
from sklearn.metrics import *
from sklearn.cluster import *
import numpy as np


In [37]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, stop_words = stopwords_rus, analyzer = 'word')),
    ('tfidf', TfidfTransformer()),
    #('svd', TruncatedSVD(n_components = 1500)),
    #('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 16, random_state = 42))
])


pipeline.fit(articles_sample.text)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...=16, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

In [50]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = articles_sample.rubrics

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.322376354155
Completeness: 0.283890706408
V-measure 0.301912001702
Adjusted Rand-Index: 0.119989066892


In [51]:
labels.groupby(clust_labels).value_counts()

    rubrics            
0   Космос                 38
    Технологии             25
    Computer Science       23
    Футурология            13
    Науки о земле          10
    Физика                  9
    Биология                5
    Социология              2
    Экономика               2
    Язык                    2
    Математика              1
    Физиология человека     1
1   Биология               70
    Технологии             52
    Психология             39
    Computer Science       35
    Физиология человека    22
    Физика                 20
    Язык                   19
    Социология             14
    Математика             13
    История                11
    Футурология             8
    Космос                  6
    Химия                   5
    Культура                4
    Науки о земле           2
    Экономика               2
2   Космос                 21
    Физика                  5
                           ..
13  Биология               41
    Социология  

In [54]:
articles_sample2 = articles[(articles.rubrics != 'Мусор')&(articles.rubrics_number == 'One')].head(2000)

In [55]:
texts = []
for path in tqdm(articles_sample2.path):
    path = path.replace('\n','').replace('//','/').replace('?code=','-code=')
    try:
        with open('C:/Users/Анна/YandexDisk/popular_science_texts_store/' + path, encoding = 'utf-8') as f:
            texts.append(f.read())
    except OSError:
        texts.append('')

100%|████████████████████████████████████| 2000/2000 [00:01<00:00, 1340.43it/s]


In [56]:
articles_sample2['text'] = texts

In [58]:
articles_sample2.text = preprocess(articles_sample2.text)

100%|█████████████████████████████████████| 2000/2000 [00:02<00:00, 909.06it/s]


In [60]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, stop_words = stopwords_rus, analyzer = 'word')),
    ('tfidf', TfidfTransformer()),
    #('svd', TruncatedSVD(n_components = 1500)),
    #('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 16, random_state = 42))
])


pipeline.fit(articles_sample2.text)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...=16, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

In [66]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = articles_sample2.rubrics

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.379048522594
Completeness: 0.334265418801
V-measure 0.355251189688
Adjusted Rand-Index: 0.176918791332


In [64]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, analyzer = 'word', stop_words = stopwords_rus)),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 1500)),
    ('norm', Normalizer() ),
    ('clust', AgglomerativeClustering(n_clusters = 16, affinity = 'euclidean')) 
])

pipeline2.fit(articles_sample2.text)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...ory(cachedir=None), n_clusters=16,
            pooling_func=<function mean at 0x00000000050F17B8>))])

In [65]:
clust_labels = pipeline2.named_steps['clust'].labels_
labels = articles_sample2.rubrics

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.339795337239
Completeness: 0.360931411627
V-measure 0.350044609921
Adjusted Rand-Index: 0.17576233936
